# Análise de Comportamento de Clientes para E-commerce
---

## 1. Introdução e Objetivos

Este notebook documenta a análise completa do dataset "E-commerce Customer Behavior", desde a limpeza dos dados brutos até a formulação de recomendações estratégicas.

#### Objetivo da Análise
Analisar o comportamento dos clientes para identificar o perfil dos consumidores de maior valor (`Total Spend`) e os principais fatores que influenciam a satisfação e a retenção (`Satisfaction Level`, `Days Since Last Purchase`).

#### Perguntas-Chave
1.  **Valor do Cliente:** Qual é o perfil (idade, gênero, cidade, tipo de perfil) dos clientes que mais gastam?
2.  **Retenção:** Existe uma relação entre o tempo desde a última compra e o nível de satisfação?
3.  **Satisfação:** O uso de descontos tem impacto no nível de satisfação e no valor gasto pelo cliente?

---

## 2. Limpeza e Preparação dos Dados

A primeira e mais crucial etapa é a limpeza dos dados. O script a seguir foi desenvolvido para carregar o arquivo `customer.csv` bruto, corrigir os nomes das colunas, tratar formatos numéricos inconsistentes, preencher valores ausentes específicos e, finalmente, exportar um novo arquivo limpo chamado `customer_limpo.csv` para ser usado na análise.

In [1]:
"""
Script para limpeza e preparação inicial do dataset de comportamento de clientes de e-commerce.
"""

# 1. Importar a biblioteca Pandas
import pandas as pd
import numpy as np # Import numpy for NaN values

# --- CONFIGURAÇÃO ---
file_path = 'customer.csv'

# 2. Carregar o Dataset
try:
    # O separador padrão ',' é o correto para a leitura deste arquivo.
    df = pd.read_csv(file_path, sep=',')
    print("Arquivo CSV carregado com sucesso!\n")
except FileNotFoundError:
    print(f"Erro: O arquivo '{file_path}' não foi encontrado.")
    print("Por favor, verifique se o nome do arquivo está correto e considere usar o caminho absoluto (completo) do arquivo.")


# 3. Limpeza e Formatação das Colunas
print("--- 3. Renomeando Colunas ---")
print("Nomes das colunas originais:")
print(df.columns)

# Renomeia todas as colunas para o padrão "snake_case" (minúsculas e com underscores)
df.columns = df.columns.str.replace(' ', '_').str.lower()

print("\nNomes das colunas formatados:")
print(df.columns)
print("\n")


# 4. Conversão de Tipos de Dados (Lidando com Formato Numérico Brasileiro/Europeu)
print("--- 4. Corrigindo Tipos de Dados Numéricos ---")

# Lista de colunas que podem estar como texto devido à formatação com vírgula
numeric_cols_to_clean = ['total_spend', 'average_rating']

for col in numeric_cols_to_clean:
    if col in df.columns and df[col].dtype == 'object':
        print(f"Processando a coluna '{col}'...")
        # Garante que a coluna é do tipo string para usar os métodos .str
        df[col] = df[col].astype(str)
        # Passo A: Remover o ponto de milhar (ex: '1.500,50' -> '1500,50')
        df[col] = df[col].str.replace('.', '', regex=False)
        # Passo B: Substituir a vírgula de decimal por um ponto (ex: '1500,50' -> '1500.50')
        df[col] = df[col].str.replace(',', '.', regex=False)
        # Passo C: Converter a coluna para o tipo numérico (float)
        df[col] = pd.to_numeric(df[col], errors='coerce')

print("\nTipos de dados após a conversão:")
df.info()
print("\n")


# 5. Tratamento de Valores Ausentes Específicos
print("--- 5. Tratando Valores Ausentes Específicos ---")
# Preenche os valores em branco na coluna 'satisfaction_level' para clientes específicos.
df.loc[df['customer_id'].isin([172, 244]), 'satisfaction_level'] = 'Neutral'

# Remove espaços em branco de outras entradas na coluna, caso existam
if 'satisfaction_level' in df.columns:
    df['satisfaction_level'] = df['satisfaction_level'].str.strip()
    # Preenche qualquer valor que ficou vazio após o strip
    df['satisfaction_level'].replace('', np.nan, inplace=True)

print("Valores de 'satisfaction_level' para os clientes 172 e 244 foram preenchidos.\n")


# 6. Verificação Geral de Valores Ausentes
print("--- 6. Verificação Geral de Valores Ausentes ---")
missing_values = df.isnull().sum()
print(missing_values)
print("\n")

if missing_values.sum() == 0:
    print("Ótima notícia! Não há mais valores ausentes no dataset.")
else:
    print("Atenção: Ainda existem valores ausentes. Pode ser necessário um tratamento adicional.")

# 7. Exibindo as primeiras linhas do DataFrame limpo
print("\n--- Amostra dos Dados Limpos (Primeiras 5 Linhas) ---")
print(df.head())


# 8. Exportar o DataFrame Limpo para Power BI (Formato Brasileiro/Europeu)
# Salva o DataFrame final em um novo arquivo CSV, formatado para ser lido corretamente por Power BI em regiões que usam vírgula como decimal.
# sep=';' -> Usa ponto e vírgula como separador de colunas para evitar conflito com a vírgula do decimal.
# decimal=',' -> Usa a vírgula como separador decimal (ex: 1120,2).
# index=False -> Evita que o Pandas escreva o índice do DataFrame como uma coluna no arquivo.
output_filename = 'customer_limpo.csv'
df.to_csv(output_filename, index=False, sep=';', decimal=',')

print(f"\n--- Exportação Concluída ---")
print(f"O DataFrame limpo foi salvo com sucesso no arquivo: '{output_filename}'")
print("O arquivo foi formatado com ';' como separador de colunas e ',' como separador decimal.")


Erro: O arquivo 'customer.csv' não foi encontrado.
Por favor, verifique se o nome do arquivo está correto e considere usar o caminho absoluto (completo) do arquivo.
--- 3. Renomeando Colunas ---
Nomes das colunas originais:


NameError: name 'df' is not defined

## 3. Análise Exploratória de Dados (EDA)

Com os dados limpos e salvos em `customer_limpo.csv`, podemos agora carregá-los e iniciar a análise exploratória.

In [3]:
# Carregar o dataset limpo para a análise
file_path_limpo = 'src/customer_limpo.csv'
try:
    # Usamos sep=';' e decimal=',' para ler o arquivo formatado corretamente
    df_limpo = pd.read_csv(file_path_limpo, sep=';', decimal=',')
    print("Dataset limpo carregado com sucesso!")
    print(f"O dataset contém {df_limpo.shape[0]} linhas e {df_limpo.shape[1]} colunas.")
    print("\n--- Informações Gerais do DataFrame Limpo ---")
    df_limpo.info()
except FileNotFoundError:
    print(f"Erro: Arquivo '{file_path_limpo}' não encontrado.")
    df_limpo = pd.DataFrame()

if not df_limpo.empty:
    # --- 3.1 Análise de Satisfação vs. Retenção ---
    # Hipótese: A satisfação do cliente tem um impacto direto na sua retenção.
    # Conclusão: Os dados confirmam a hipótese. Clientes "Unsatisfied" demoram, em média, 
    # muito mais tempo para retornar do que clientes "Neutral" ou "Satisfied".
    
    print("\n--- Média de dias desde a última compra por Nível de Satisfação ---")
    retencao_por_satisfacao = df_limpo.groupby('satisfaction_level')['days_since_last_purchase'].mean().sort_values(ascending=False).reset_index()
    display(retencao_por_satisfacao)

    # --- 3.2 Análise Geográfica de Valor do Cliente ---
    # Objetivo: Investigar onde estão nossos clientes mais valiosos.
    # Conclusão: A análise mostra uma disparidade enorme. San Francisco e New York geram a maior parte da receita, 
    # apesar de terem um número de clientes similar às outras cidades. O desafio não é aquisição, mas o desenvolvimento do valor do cliente.
    
    print("\n--- Análise de Valor por Cidade ---")
    valor_por_cidade = df_limpo.groupby('city').agg(
        total_clientes=('customer_id', 'count'),
        total_gasto=('total_spend', 'sum'),
        media_itens_comprados=('items_purchased', 'mean')
    ).sort_values(by='total_gasto', ascending=False)
    display(valor_por_cidade)

    # --- 3.3 Análise da Estratégia de Descontos ---
    # Objetivo: Verificar a eficácia da estratégia de descontos.
    # Conclusão: Os dados revelam um paradoxo. Em Chicago, 100% dos clientes recebem desconto, mas apresentam 
    # menor gasto e menor satisfação, enquanto em San Francisco ocorre o oposto.
    
    print("\n--- Comparação da Estratégia de Descontos (San Francisco vs. Chicago) ---")
    comparacao_descontos = df_limpo[df_limpo['city'].isin(['San Francisco', 'Chicago'])].groupby('city').agg(
        media_gasto=('total_spend', 'mean'),
        media_avaliacao=('average_rating', 'mean'),
        percentual_com_desconto=('discount_applied', lambda x: x.mean() * 100)
    )
    display(comparacao_descontos)

Dataset limpo carregado com sucesso!
O dataset contém 350 linhas e 11 colunas.

--- Informações Gerais do DataFrame Limpo ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               350 non-null    int64  
 1   gender                    350 non-null    object 
 2   age                       350 non-null    int64  
 3   city                      350 non-null    object 
 4   membership_type           350 non-null    object 
 5   total_spend               350 non-null    float64
 6   items_purchased           350 non-null    int64  
 7   average_rating            350 non-null    float64
 8   discount_applied          350 non-null    bool   
 9   days_since_last_purchase  350 non-null    int64  
 10  satisfaction_level        350 non-null    object 
dtypes: bool(1), float64(2), int64(4), object(4)
memor

,satisfaction_level,days_since_last_purchase
0,Unsatisfied,42.982759
1,Neutral,19.339450
2,Satisfied,17.696000



--- Análise de Valor por Cidade ---


,total_clientes,total_gasto,media_itens_comprados
city,,,
San Francisco,58,84666.8,20.000000
New York,59,68737.1,15.271186
Los Angeles,59,47524.0,11.677966
Miami,58,40042.6,11.637931
Chicago,58,28993.2,9.413793
Houston,58,25919.9,7.568966



--- Comparação da Estratégia de Descontos (San Francisco vs. Chicago) ---


,media_gasto,media_avaliacao,percentual_com_desconto
city,,,
Chicago,499.882759,3.456897,100.0
San Francisco,1459.772414,4.808621,0.0


In [4]:
# --- 3.4 Análise do Ticket Médio por Cidade ---
# Objetivo: Entender se clientes em cidades de alto valor gastam mais por item comprado.
# Conclusão: Sim, o valor médio por item em NY e San Francisco é significativamente maior, 
# o que sugere que esses clientes compram produtos de maior valor agregado.

if not df_limpo.empty:
    print("\n--- Análise de Ticket Médio por Cidade ---")
    
    # Primeiro, vamos recalcular os totais para ter todos os dados necessários
    analise_cidade = df_limpo.groupby('city').agg(
        total_gasto=('total_spend', 'sum'),
        total_itens=('items_purchased', 'sum')
    )
    
    # Calcular o ticket médio (valor gasto por item)
    analise_cidade['ticket_medio_por_item'] = analise_cidade['total_gasto'] / analise_cidade['total_itens']
    
    display(analise_cidade.sort_values(by='ticket_medio_por_item', ascending=False))


# --- 3.5 Análise Demográfica (Gênero e Idade) ---
# Objetivo: Identificar o perfil demográfico dos clientes de maior valor.
# Conclusão: O público masculino representa a grande maioria do faturamento. 
# A faixa etária mais valiosa é a de 26 a 35 anos, que sozinha responde por mais da metade da receita total.

if not df_limpo.empty:
    print("\n--- Análise de Faturamento por Gênero ---")
    faturamento_genero = df_limpo.groupby('gender').agg(
        total_clientes=('customer_id', 'count'),
        total_gasto=('total_spend', 'sum')
    ).sort_values(by='total_gasto', ascending=False)
    
    faturamento_genero['percentual_gasto'] = (faturamento_genero['total_gasto'] / df_limpo['total_spend'].sum()) * 100
    display(faturamento_genero)

    print("\n--- Análise de Faturamento por Faixa Etária ---")
    # Criar faixas etárias para agrupar os clientes
    bins = [18, 25, 35, 45, 55, 65, 100]
    labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '65+']
    df_limpo['faixa_etaria'] = pd.cut(df_limpo['age'], bins=bins, labels=labels, right=False)
    
    faturamento_idade = df_limpo.groupby('faixa_etaria').agg(
        total_clientes=('customer_id', 'count'),
        total_gasto=('total_spend', 'sum')
    ).sort_values(by='total_gasto', ascending=False)
    
    faturamento_idade['percentual_gasto'] = (faturamento_idade['total_gasto'] / df_limpo['total_spend'].sum()) * 100
    display(faturamento_idade)


--- Análise de Ticket Médio por Cidade ---


,total_gasto,total_itens,ticket_medio_por_item
city,,,
New York,68737.1,901,76.289789
San Francisco,84666.8,1160,72.988621
Los Angeles,47524.0,689,68.975327
Miami,40042.6,675,59.322370
Houston,25919.9,439,59.043052
Chicago,28993.2,546,53.101099



--- Análise de Faturamento por Gênero ---


,total_clientes,total_gasto,percentual_gasto
gender,,,
Male,175,172713.6,58.372144
Female,175,123170.0,41.627856



--- Análise de Faturamento por Faixa Etária ---


C:\Users\gabri\AppData\Local\Temp\ipykernel_19920\4111033628.py:42: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  faturamento_idade = df_limpo.groupby('faixa_etaria').agg(


,total_clientes,total_gasto,percentual_gasto
faixa_etaria,,,
26-35,209,219998.1,74.352921
36-45,141,75885.5,25.647079
18-25,0,0.0,0.000000
46-55,0,0.0,0.000000
56-65,0,0.0,0.000000
65+,0,0.0,0.000000


## 4. Visualização de Dados e Recomendações Estratégicas

Para aprofundar nossa análise, criamos uma série de visuais no Power BI. Cada gráfico nos ajudou a responder uma pergunta específica e a construir nossas recomendações finais, transformando os dados em uma história visual.

### Análise 1: A Satisfação como Motor da Retenção

A primeira pergunta que investigamos foi a relação entre a satisfação do cliente e sua lealdade. O gráfico de barras abaixo mostra a média de dias desde a última compra para cada nível de satisfação.

![Gráfico de Retenção por Satisfação](src/retencao_satisfacao.png)

* **Insight:** O gráfico prova de forma clara que a **insatisfação é o principal indicador de abandono**. Clientes "Unsatisfied" demoram significativamente mais tempo para voltar a comprar, o que indica um alto risco de churn. A proximidade entre "Neutral" e "Satisfied" sugere que a empresa precisa focar não apenas em evitar a insatisfação, mas em ativamente encantar o cliente para maximizar a retenção.

---

### Análise 2: A Geografia do Valor do Cliente

A seguir, investigamos onde estão nossos clientes mais valiosos. O gráfico de colunas e linhas revela uma descoberta surpreendente:

![Gráfico de Faturamento vs. Quantidade de Clientes por Cidade](src/faturamento_qtde_clientes_cidade.png)

Embora o número de clientes (linha azul) seja praticamente o mesmo em todas as cidades, o faturamento (barras) é drasticamente diferente. O mapa de calor reforça essa concentração de valor:

![Mapa de Faturamento por Cidade](src/mapa.png)

Para entender melhor essa diferença, analisamos o "ticket médio por item" em cada cidade.

![Ticket Médio por Cidade](src/ticket_cidade.png)

* **Insight:** Juntos, estes gráficos mostram que o **desafio da empresa não é a aquisição de clientes, mas sim o desenvolvimento do valor** em cada mercado. Clientes em San Francisco e Nova York não só gastam mais no total, como também compram produtos de maior valor (ticket médio superior).

---

### Análise 3: O Paradoxo dos Descontos e o Perfil Demográfico

Finalmente, investigamos a eficácia da estratégia de descontos e o perfil demográfico dos clientes.

![Tabela Comparativa de Descontos](src/tabela_desconto.png)

* **Insight:** A estratégia de descontos atual é ineficaz. Chicago, o mercado de menor valor, recebe 100% de desconto, enquanto San Francisco, o de maior valor, não recebe nenhum. Isso sugere que os descontos podem estar a prejudicar a percepção de valor da marca.

![Faturamento por Idade e Gênero](src/faturamento_idade_genero.png)

* **Insight:** O perfil do cliente de maior valor começa a tomar forma: a análise demográfica mostra que o público **masculino** e a faixa etária próxima aos **30 anos** são os que mais geram faturamento para a empresa.

---

### Conclusões e Recomendações Estratégicas Finais

1.  **Foco na Experiência do Cliente:** Implementar um sistema proativo para monitorizar a satisfação e intervir rapidamente para reverter experiências negativas, especialmente no "momento da verdade" dos clientes Silver.
2.  **Revisão da Estratégia de Descontos:** Testar a remoção de descontos em mercados de baixo desempenho (como Chicago) e reinvestir o orçamento em estratégias que aumentem o valor percebido da marca, replicando o modelo de sucesso de San Francisco.
3.  **Marketing Direcionado:** Direcionar os esforços de aquisição para o perfil de cliente ideal identificado: homens, com cerca de 30 anos, residentes em cidades de alto potencial como Nova York e San Francisco.